## Se de utilizar una versión de Python entre la 3.8.1 y menor a la 3.12 para utilizar LlamaIndex

In [ ]:
!pip install -q llama-index==0.9.1 chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.2/874.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.response.notebook_utils import display_response
import os
from getpass import getpass

 ## Key

In [ ]:
OPENAI_API_KEY = getpass('Enter the secret value for OpenAI Key: ')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter the secret value for OpenAI Key: ··········


Para probar que se concectado correctamente a la API de OpenAI

In [ ]:
resp = OpenAI().complete('Que es InnovaLAB UFRO?')
print(resp)

InnovaLAB UFRO es un laboratorio de innovación y emprendimiento de la Universidad de La Frontera (UFRO) en Chile. Este laboratorio tiene como objetivo fomentar la creatividad, la innovación y el emprendimiento entre los estudiantes, académicos y la comunidad en general, a través de la realización de actividades, talleres, charlas y programas de apoyo para el desarrollo de proyectos innovadores. InnovaLAB UFRO busca ser un espacio de colaboración y aprendizaje donde se puedan generar soluciones creativas a problemas actuales y promover el espíritu emprendedor en la comunidad universitaria.


RAG con Llama index

In [ ]:
### Cargando documentos
documents = SimpleDirectoryReader('Input').load_data()

In [ ]:
documents

[Document(id_='95f34bd5-5bda-4cb5-ab25-0d7c311b677f', embedding=None, metadata={'file_path': 'Input/Prueba_causa.json', 'file_name': 'Prueba_causa.json', 'file_type': 'application/json', 'file_size': 6574, 'creation_date': '2024-06-19', 'last_modified_date': '2024-06-19', 'last_accessed_date': '2024-06-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='d52c4685fff10cbc697b09fa1815522dd7620a5abf56aedf6288c2800967b9b9', text='[\n    {\n      "court": "C.A. de Temuco",\n      "role": "38364",\n      "year": "2022",\n      "text": "POR TANTO: De acuerdo con lo dispuesto en los artículos 19, números 1°, 9° y 24°; artículo 20 de la Constitución Política de la República y al Auto Acordado de la Excma. Corte Suprema, sobre tramitación y fallo del recurso de p

In [ ]:
prompt="""Analiza los textos proporcionados para identificar y comparar casos judiciales relacionados con el alza de precios de planes de isapres. Concéntrate en los siguientes elementos:
        1. Identificar el nombre del recurrente, que aparece después de "a favor de".
        2. Identificar el nombre del recurrido, que aparece después de "en contra de".
        3. Extraer la materia, que se encuentra entre "Corte Suprema, sobre tramitación y fallo del" y "acto que menoscaba".
        4. Formatear la caratulada como "apellidos recurrente/recurrido" en mayúsculas.
        5. Comparar los textos para determinar similitudes y diferencias en los elementos mencionados.

        Debes devolver la respuesta exclusivamente en formato JSON como un arreglo de objetos con la siguiente estructura:
        {{
        "solicitant_name": "string",
        "respondent_name": "string",
        "subject_matter": "string",
        "case_heading": "string",
        "similarity_score": "float",
        "similar_text": "string"
        "difference_text":"string"
        }}
        No incluyas texto adicional fuera del JSON."""


In [ ]:
### Creando un indice con base a los documentos
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
## Ejecutando inferencia con Rag
response = query_engine.query(prompt)
#display_response(response)

In [ ]:
print(response)

{
    "solicitant_name": "BARTOLOMÉ ENRIQUE SAGREDO COLL",
    "respondent_name": "ISAPRE Consalud S.A.",
    "subject_matter": "recurso de protección de garantías constitucionales",
    "case_heading": "SAGREDO COLL/CONSALUD S.A.",
    "similarity_score": 0.75,
    "similar_text": "POR TANTO: De acuerdo con lo dispuesto en los artículos 19, números 1°, 9° y 24°; artículo 20 de la Constitución Política de la República y al Auto Acordado de la Excma. Corte Suprema, sobre tramitación y fallo del recurso de protección de garantías constitucionales, y demás disposiciones constitucionales y legales citadas,  SOLICITO A V.S.I. que tenga por presentado este recurso de protección a favor de BARTOLOMÉ ENRIQUE SAGREDO COLL, en contra de ISAPRE Consalud S.A., por los actos ilegales y arbitrarios en que incurre, al haber modificado unilateralmente mi plan individual de salud, al alzar el precio de la cobertura GES, acto que menoscaba los derechos constitucionales asegurados en los números 1º, 9º y

In [ ]:
type(response)

llama_index.response.schema.Response

In [ ]:
response_str = str(response)
response_str = response_str.replace('\n', '')  # Eliminar saltos de línea
print(type(response_str))  # Debería imprimir <class 'str'>
print(response_str)  # Imprimir la respuesta como string sin saltos de línea

<class 'str'>
{    "solicitant_name": "BARTOLOMÉ ENRIQUE SAGREDO COLL",    "respondent_name": "ISAPRE Consalud S.A.",    "subject_matter": "recurso de protección de garantías constitucionales",    "case_heading": "SAGREDO COLL/CONSALUD S.A.",    "similarity_score": 0.75,    "similar_text": "POR TANTO: De acuerdo con lo dispuesto en los artículos 19, números 1°, 9° y 24°; artículo 20 de la Constitución Política de la República y al Auto Acordado de la Excma. Corte Suprema, sobre tramitación y fallo del recurso de protección de garantías constitucionales, y demás disposiciones constitucionales y legales citadas,  SOLICITO A V.S.I. que tenga por presentado este recurso de protección a favor de BARTOLOMÉ ENRIQUE SAGREDO COLL, en contra de ISAPRE Consalud S.A., por los actos ilegales y arbitrarios en que incurre, al haber modificado unilateralmente mi plan individual de salud, al alzar el precio de la cobertura GES, acto que menoscaba los derechos constitucionales asegurados en los números 

In [ ]:
response

Response(response='{\n    "solicitant_name": "BARTOLOMÉ ENRIQUE SAGREDO COLL",\n    "respondent_name": "ISAPRE Consalud S.A.",\n    "subject_matter": "recurso de protección de garantías constitucionales",\n    "case_heading": "SAGREDO COLL/CONSALUD S.A.",\n    "similarity_score": 0.75,\n    "similar_text": "POR TANTO: De acuerdo con lo dispuesto en los artículos 19, números 1°, 9° y 24°; artículo 20 de la Constitución Política de la República y al Auto Acordado de la Excma. Corte Suprema, sobre tramitación y fallo del recurso de protección de garantías constitucionales, y demás disposiciones constitucionales y legales citadas,  SOLICITO A V.S.I. que tenga por presentado este recurso de protección a favor de BARTOLOMÉ ENRIQUE SAGREDO COLL, en contra de ISAPRE Consalud S.A., por los actos ilegales y arbitrarios en que incurre, al haber modificado unilateralmente mi plan individual de salud, al alzar el precio de la cobertura GES, acto que menoscaba los derechos constitucionales asegurado

In [ ]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='fa372ff5-430c-4f36-bd06-cd7ac36fb3f9', embedding=None, metadata={'file_path': 'Input/Prueba_causa.json', 'file_name': 'Prueba_causa.json', 'file_type': 'application/json', 'file_size': 6574, 'creation_date': '2024-06-19', 'last_modified_date': '2024-06-19', 'last_accessed_date': '2024-06-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='95f34bd5-5bda-4cb5-ab25-0d7c311b677f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'Input/Prueba_causa.json', 'file_name': 'Prueba_causa.json', 'file_type': 'application/json', 'file_size': 6574, 'creation_date': '2024-06-19', 'last_modified_date': '2024-06-19', 'last_accessed_date': '2024-06-19'}, hash='d52c4685fff10cbc

In [ ]:
for _ in response.source_nodes:
    print(_.text)

[
    {
      "court": "C.A. de Temuco",
      "role": "38364",
      "year": "2022",
      "text": "POR TANTO: De acuerdo con lo dispuesto en los artículos 19, números 1°, 9° y 24°; artículo 20 de la Constitución Política de la República y al Auto Acordado de la Excma. Corte Suprema, sobre tramitación y fallo del recurso de protección de garantías constitucionales, y demás disposiciones constitucionales y legales citadas,  SOLICITO A V.S.I. que tenga por presentado este recurso de protección a favor de BARTOLOMÉ ENRIQUE SAGREDO COLL, en contra de ISAPRE Consalud S.A., por los actos ilegales y arbitrarios en que incurre, al haber modificado unilateralmente mi plan individual de salud, al alzar el precio de la cobertura GES, acto que menoscaba los derechos constitucionales asegurados en los números 1º, 9º y 24º del artículo 19 de la Constitución Política de la República, acogerlo a tramitación y en definitiva declarar:  1.- Que el alza a 1,390 UF por cada carga inscrita, en el precio de

 Valores default - service_context

In [ ]:
index.service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=OpenAIEmbedding(model_name='text-embedding-ada-002', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ade3d10d300>, additional_kwargs={}, api_key='sk-proj-baUeBIj0g0DOpKfxHjEFT3BlbkFJT3Yt0UKh6vgcqsPLVIF0', api_base='https://api.openai.com/v1', api_version='', max_retries=10, timeout=60.0), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ade3d10d300>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7ad

In [ ]:
index.service_context.llm

OpenAI(callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ade3d10d300>, model='gpt-3.5-turbo', temperature=0.1, max_tokens=None, additional_kwargs={}, max_retries=3, timeout=60.0, api_key='sk-proj-baUeBIj0g0DOpKfxHjEFT3BlbkFJT3Yt0UKh6vgcqsPLVIF0', api_base='https://api.openai.com/v1', api_version='')

In [ ]:
index.service_context.embed_model

OpenAIEmbedding(model_name='text-embedding-ada-002', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ade3d10d300>, additional_kwargs={}, api_key='sk-proj-baUeBIj0g0DOpKfxHjEFT3BlbkFJT3Yt0UKh6vgcqsPLVIF0', api_base='https://api.openai.com/v1', api_version='', max_retries=10, timeout=60.0)

In [ ]:
index.service_context.transformations

[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ade3d10d300>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')]


Valores default - service_context

In [ ]:
index.storage_context

StorageContext(docstore=<llama_index.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x7ade3d10d510>, index_store=<llama_index.storage.index_store.simple_index_store.SimpleIndexStore object at 0x7ade3d10d690>, vector_stores={'default': <llama_index.vector_stores.simple.SimpleVectorStore object at 0x7ade3d10d4e0>}, graph_store=<llama_index.graph_stores.simple.SimpleGraphStore object at 0x7ade3d10d5a0>)

In [ ]:
index.storage_context.vector_stores

{'default': <llama_index.vector_stores.simple.SimpleVectorStore at 0x7ade3d10d4e0>}